In [2]:
import os

os.chdir(r"/home/kumail/Quantized-RAG")

# https://python.langchain.com/v0.1/docs/integrations/vectorstores/faiss/

In [5]:
from transformers import AutoTokenizer, AutoModel
import torch
import faiss
import numpy as np
import pickle

# Load model and tokenizer
model_name = "sentence-transformers/all-MiniLM-L6-v2"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModel.from_pretrained(model_name)

# Function to compute embeddings
def compute_embeddings(texts):
    inputs = tokenizer(texts, padding=True, truncation=True, return_tensors="pt")
    with torch.no_grad():
        model_output = model(**inputs)
    embeddings = model_output.last_hidden_state.mean(dim=1)  # mean pooling
    return embeddings.numpy()

In [6]:
import joblib

documents = joblib.load('data/processed_text.pkl')

In [10]:
# Compute embeddings
document_embeddings = compute_embeddings(documents)

# Build FAISS index
dimension = document_embeddings.shape[1]
index = faiss.IndexFlatL2(dimension)
index.add(document_embeddings)

# Save FAISS index and documents
index_file = "data/faiss_index.bin"
faiss.write_index(index, index_file)

documents_file = "data/documents.pkl"
with open(documents_file, 'wb') as f:
    pickle.dump(documents, f)

In [11]:
document_embeddings

len(document_embeddings)

1163

In [3]:
from transformers import AutoTokenizer, AutoModel
import torch
import faiss
import numpy as np
import pickle

# Load model and tokenizer
model_name = "sentence-transformers/all-MiniLM-L6-v2"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModel.from_pretrained(model_name)

# Function to compute embeddings
def compute_embeddings(texts):
    inputs = tokenizer(texts, padding=True, truncation=True, return_tensors="pt")
    with torch.no_grad():
        model_output = model(**inputs)
    embeddings = model_output.last_hidden_state.mean(dim=1)  # mean pooling
    return embeddings.numpy()



documents_file = "data/documents.pkl"
index_file = "data/faiss_index.bin"


# Load FAISS index and documents
index = faiss.read_index(index_file)
with open(documents_file, 'rb') as f:
    documents = pickle.load(f)





In [4]:
# Perform a search query
query = "how apple is performing "
query_embedding = compute_embeddings([query])

k = 3  # Number of nearest neighbors to retrieve
distances, indices = index.search(query_embedding, k)

# Display results
print("Results:")
for i, idx in enumerate(indices[0]):
    print(f"Rank {i+1}:")
    print(f"Document: {documents[idx]}")
    print(f"Distance: {distances[0][i]}")


Results:
Rank 1:
Document: Apple Inc.'s total revenue in 2022 was $394.33 billion.
Distance: 43.733909606933594
Rank 2:
Document: Apple's Total Liabilities for the TTM period are 50.80 billion USD.
Distance: 47.49530029296875
Rank 3:
Document: Apple's total current assets for the TTM period are 79.26 billion USD.
Distance: 48.00399398803711


## Vector Search

In [ ]:
class Vector:
    def __init__(self, model_name, index_file, documents_file) :
        self.embed_tokenizer, self.embed_model=self.load_model(model_name)
        self.faiss_index = self.load_index(index_file)
        self.embed_docs = self.load_embedded_documents(documents_file)

    def load_model(self, model_name):
        tokenizer = AutoTokenizer.from_pretrained(model_name)
        model = AutoModel.from_pretrained(model_name)
        return tokenizer, model
    
    def load_index(self, index_file):
        return faiss.read_index(index_file)
    
    def load_embedded_documents(self, documents_file):
        with open(documents_file, 'rb') as f:
            documents = pickle.load(f)
            return documents
        
    def compute_embeddings(self, texts):
        inputs = self.embed_tokenizer(texts, padding=True, truncation=True, return_tensors="pt")
        with torch.no_grad():
            model_output = model(**inputs)
        embeddings = model_output.last_hidden_state.mean(dim=1)  # mean pooling
        return embeddings.numpy()

    def cosine_search(self, query_embedding, k):
        distances, indices = index.search(query_embedding, k)
        return indices
    
    def search(self, text, k):
        query_embedding = self.compute_embeddings([text])
        retrived_indexes = self.cosine_search(query_embedding, k)
        return [self.embed_docs[i] for i in retrived_indexes[0]]


In [ ]:
model_name = "sentence-transformers/all-MiniLM-L6-v2"
documents_file = "data/documents.pkl"
index_file = "data/faiss_index.bin"

vec = Vector(model_name=model_name,
                index_file=index_file,
                documents_file=documents_file
            )